In [1]:
from langchain_community.retrievers import WikipediaRetriever

#### WikipediaRetriever

In [3]:
# Initialize the retriever (optional: set language and top_k)
retriever = WikipediaRetriever(top_k_results=2, lang="en")

In [4]:

# Define your query
query = "the geopolitical history of india and pakistan from the perspective of a chinese"

# Get relevant Wikipedia documents
docs = retriever.invoke(query)

In [6]:
docs

[Document(metadata={'title': 'Foreign relations of Pakistan', 'summary': "The Islamic Republic of Pakistan emerged as an independent country through the partition of India in August 1947 and was admitted as a United Nations member state in September 1947. It is currently the second-most populous country within the Muslim world, and is also the only Muslim-majority country openly in possession of nuclear weapons. The country shares land borders with India, Iran, Afghanistan, and China.\nThe country has extensive trade relations with the European Union and with several countries globally. As of 2023, Pakistan does not recognize two other United Nations member states (Armenia and Israel) and its ties with India remain frozen since 2019.\nFrom a geopolitical perspective, Pakistan's location is strategically important as it is situated at the crossroads of major maritime and land transit routes between the Middle East and South Asia, while also serving as a bridge between the Arabian Sea an

In [7]:
# Print retrieved content
for i, doc in enumerate(docs):
    print(f"\n--- Result {i+1} ---")
    print(f"Content:\n{doc.page_content}...")  # truncate for display


--- Result 1 ---
Content:
The Islamic Republic of Pakistan emerged as an independent country through the partition of India in August 1947 and was admitted as a United Nations member state in September 1947. It is currently the second-most populous country within the Muslim world, and is also the only Muslim-majority country openly in possession of nuclear weapons. The country shares land borders with India, Iran, Afghanistan, and China.
The country has extensive trade relations with the European Union and with several countries globally. As of 2023, Pakistan does not recognize two other United Nations member states (Armenia and Israel) and its ties with India remain frozen since 2019.
From a geopolitical perspective, Pakistan's location is strategically important as it is situated at the crossroads of major maritime and land transit routes between the Middle East and South Asia, while also serving as a bridge between the Arabian Sea and the energy-rich regions of Central Asia. Since 

#### Vector Store Retriever

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from dotenv import load_dotenv
import os

In [5]:
# Load environment variables from the .env file
load_dotenv()

# # Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [6]:
# Step 1: Your source documents
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [7]:
# Initialize the embeddings object
gemini_embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=gemini_api_key
)

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

gemini_model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=gemini_api_key, temperature=0.3)


In [12]:
vector_store = Chroma.from_documents(
    embedding=gemini_embeddings,
    documents=documents,
    persist_directory='retriever_demo_vector_store',
    collection_name='sample_retriever_demo'
)

In [13]:
# Step 4: Convert vectorstore into a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

In [14]:
query = "What is Chroma used for?"
results = retriever.invoke(query)

In [15]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Chroma is a vector database optimized for LLM-based search.

--- Result 2 ---
Embeddings convert text into high-dimensional vectors.


#### MMR (Maximum Marginal Relevance)

In [16]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [17]:
from langchain_community.vectorstores import FAISS


# Step 2: Create the FAISS vector store from documents
faiss_vector_store = FAISS.from_documents(
    documents=docs,
    embedding=gemini_embeddings
)

In [19]:
# Enable MMR in the retriever
mmr_retriever = faiss_vector_store.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [20]:
query = "What is langchain?"
results = mmr_retriever.invoke(query)

In [21]:
for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


#### Multiquery Retriver

In [22]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

In [23]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [24]:
from langchain_community.vectorstores import FAISS


# Step 2: Create the FAISS vector store from documents
faiss_vector_store = FAISS.from_documents(
    documents=all_docs,
    embedding=gemini_embeddings
)

In [25]:
# Create retrievers
similarity_retriever = faiss_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [27]:
multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=faiss_vector_store.as_retriever(search_kwargs={"k": 5}),
    llm=gemini_model
)

In [28]:
# Query
query = "How to improve energy levels and maintain balance?"

In [29]:
# Retrieve results
similarity_results = similarity_retriever.invoke(query)
multiquery_results= multiquery_retriever.invoke(query)

In [30]:
for i, doc in enumerate(similarity_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Deep sleep is crucial for cellular repair and emotional regulation.

--- Result 5 ---
Regular walking boosts heart health and can reduce symptoms of depression.
******************************************************************************************************************************************************

--- Result 1 ---
Drinking sufficient water throughout the day helps maintain metabolism and energy.

--- Result 2 ---
Mindfulness and controlled breathing lower cortisol and improve mental clarity.

--- Result 3 ---
Consuming leafy greens and fruits helps detox the body and improve longevity.

--- Result 4 ---
Regular walking boosts heart health and can reduce symptoms of

#### Contextual Compression Retriever

In [31]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document

In [32]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [33]:
from langchain_community.vectorstores import FAISS


# Step 2: Create the FAISS vector store from documents
faiss_vector_store = FAISS.from_documents(
    documents=docs,
    embedding=gemini_embeddings
)

In [34]:
base_retriever = faiss_vector_store.as_retriever(search_kwargs={"k": 5})

In [35]:
# Set up the compressor using an LLM
compressor = LLMChainExtractor.from_llm(gemini_model)

In [36]:
# Create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

In [37]:
# Query the retriever
query = "What is photosynthesis?"
compressed_results = compression_retriever.invoke(query)

KeyboardInterrupt: 

In [ ]:
for i, doc in enumerate(compressed_results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)